In [1]:
import json
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import multilabel_confusion_matrix

In [5]:
def completely_matched_tags(lst):
    return (lst[0] == lst[1]).sum()

In [6]:
def missing_tags(lst):
    return (lst[0] > lst[1]).sum()

In [7]:
def wrong_tags(lst):
    return (lst[0]<lst[1]).sum()

In [8]:
df_sample = pd.read_csv("sampled_data_with_predictions_testset.csv")

In [9]:
df_sample["project_id"].unique()

array([2028., 2098., 2170.])

In [50]:
df_sample["project_id"].value_counts()

2028.0    50
2098.0    50
2170.0    50
Name: project_id, dtype: int64

In [10]:
df_sample.columns

Index(['entry_id', 'excerpt', 'analysis_framework_id', 'lead_id', 'project_id',
       'verified', 'sectors', 'subpillars_2d', 'subpillars_1d', 'geo_location',
       'specific_needs_groups', 'severity', 'info_date', 'age', 'gender',
       'reliability', 'affected_groups_level_0', 'affected_groups_level_1',
       'affected_groups_level_2', 'affected_groups_level_3', 'source_type',
       'url', 'website', 'lang', 'translation_en', 'translation_fr',
       'translation_es', 'sectors_pred', 'subpillars_2d_pred',
       'subpillars_1d_pred', 'age_pred', 'gender_pred', 'affected_groups_pred',
       'specific_needs_groups_pred', 'severity_pred'],
      dtype='object')

In [11]:
df_sample["sectors"] = df_sample["sectors"].apply(literal_eval)
df_sample["sectors_pred"] = df_sample["sectors_pred"].apply(literal_eval)

In [12]:
df_sample["subpillars_1d"] = df_sample["subpillars_1d"].apply(literal_eval)
df_sample["subpillars_1d_pred"] = df_sample["subpillars_1d_pred"].apply(literal_eval)

In [13]:
df_sample["subpillars_2d"] = df_sample["subpillars_2d"].apply(literal_eval)
df_sample["subpillars_2d_pred"] = df_sample["subpillars_2d_pred"].apply(literal_eval)

In [14]:
df_sample.head(2)

,entry_id,excerpt,analysis_framework_id,lead_id,project_id,verified,sectors,subpillars_2d,subpillars_1d,geo_location,...,translation_fr,translation_es,sectors_pred,subpillars_2d_pred,subpillars_1d_pred,age_pred,gender_pred,affected_groups_pred,specific_needs_groups_pred,severity_pred
0,174380,"[6th september,Overall Syria] Riza said there ...",1306,40125.0,2028.0,False,[Health],[Humanitarian Conditions->Physical And Mental ...,[Casualties->Dead],['Syrian Arab Republic'],...,"[6 septembre, la Syrie globale] Riza a déclaré...","[6 de septiembre, General Siria] Riza dijo que...",[Health],[Humanitarian Conditions->Physical And Mental ...,[Casualties->Dead],[],[],[],[],['Critical']
1,224488,"[December 21, As-Suwayda] Fear of the Coronavi...",1306,46147.0,2028.0,False,"[Protection, Education]",[Humanitarian Conditions->Physical And Mental ...,[],['As-Sweida'],...,"[21 décembre, As-Suwayda] La peur du Coronavir...","[21 de diciembre, AS-SUWAYDA] El miedo al Coro...","[Education, Health]","[Humanitarian Conditions->Living Standards, Im...",[],['Children/Youth (5 to 17 years old)'],[],[],[],['Major']


In [15]:
sectors = ['Agriculture', 'Cross', 'Education', 'Health', 'Livelihoods', 'Logistics', 'Protection', 'Shelter', 'WASH', 'Food Security', 'Nutrition']

In [16]:
subpillars_1d = ["Context->Environment", "Context->Socio Cultural", "Context->Economy",
    "Context->Demography", "Context->Legal & Policy", "Context->Security & Stability",
    "Context->Politics", "Shock/Event->Type And Characteristics", "Shock/Event->Underlying/Aggravating Factors",
    "Shock/Event->Hazard & Threats", "Displacement->Type/Numbers/Movements", "Displacement->Push Factors", "Displacement->Pull Factors",
    "Displacement->Intentions", "Displacement->Local Integration", "Casualties->Injured", "Casualties->Missing", "Casualties->Dead",
    "Humanitarian Access->Relief To Population", "Humanitarian Access->Population To Relief", "Humanitarian Access->Physical Constraints",
    "Humanitarian Access->Number Of People Facing Humanitarian Access Constraints/Humanitarian Access Gaps",
    "Information And Communication->Communication Means And Preferences", "Information And Communication->Information Challenges And Barriers",
    "Information And Communication->Knowledge And Info Gaps (Pop)", "Information And Communication->Knowledge And Info Gaps (Hum)",
    "Covid-19->Cases", "Covid-19->Contact Tracing", "Covid-19->Deaths", "Covid-19->Hospitalization & Care", "Covid-19->Restriction Measures",
    "Covid-19->Testing", "Covid-19->Vaccination"]


In [17]:
subpillars_2d = ["At Risk->Number Of People At Risk", "At Risk->Risk And Vulnerabilities", "Capacities & Response->International Response", "Capacities & Response->Local Response",
        "Capacities & Response->National Response", "Capacities & Response->Number Of People Reached/Response Gaps",
        "Humanitarian Conditions->Coping Mechanisms", "Humanitarian Conditions->Living Standards",
        "Humanitarian Conditions->Number Of People In Need", "Humanitarian Conditions->Physical And Mental Well Being",
        "Impact->Driver/Aggravating Factors", "Impact->Impact On People", "Impact->Impact On Systems, Services And Networks",
        "Impact->Number Of People Affected", "Priority Interventions->Expressed By Humanitarian Staff",
        "Priority Interventions->Expressed By Population", "Priority Needs->Expressed By Humanitarian Staff", "Priority Needs->Expressed By Population"
]

In [18]:
mlb_sectors = MultiLabelBinarizer()
mlb_sectors.fit_transform([sectors])
mlb_sectors.classes_

array(['Agriculture', 'Cross', 'Education', 'Food Security', 'Health',
       'Livelihoods', 'Logistics', 'Nutrition', 'Protection', 'Shelter',
       'WASH'], dtype=object)

In [19]:
mlb_subpillars1d = MultiLabelBinarizer()
mlb_subpillars1d.fit_transform([subpillars_1d])
mlb_subpillars1d.classes_

array(['Casualties->Dead', 'Casualties->Injured', 'Casualties->Missing',
       'Context->Demography', 'Context->Economy', 'Context->Environment',
       'Context->Legal & Policy', 'Context->Politics',
       'Context->Security & Stability', 'Context->Socio Cultural',
       'Covid-19->Cases', 'Covid-19->Contact Tracing', 'Covid-19->Deaths',
       'Covid-19->Hospitalization & Care',
       'Covid-19->Restriction Measures', 'Covid-19->Testing',
       'Covid-19->Vaccination', 'Displacement->Intentions',
       'Displacement->Local Integration', 'Displacement->Pull Factors',
       'Displacement->Push Factors',
       'Displacement->Type/Numbers/Movements',
       'Humanitarian Access->Number Of People Facing Humanitarian Access Constraints/Humanitarian Access Gaps',
       'Humanitarian Access->Physical Constraints',
       'Humanitarian Access->Population To Relief',
       'Humanitarian Access->Relief To Population',
       'Information And Communication->Communication Means And Pref

In [20]:
mlb_subpillars2d = MultiLabelBinarizer()
mlb_subpillars2d.fit_transform([subpillars_2d])
mlb_subpillars2d.classes_

array(['At Risk->Number Of People At Risk',
       'At Risk->Risk And Vulnerabilities',
       'Capacities & Response->International Response',
       'Capacities & Response->Local Response',
       'Capacities & Response->National Response',
       'Capacities & Response->Number Of People Reached/Response Gaps',
       'Humanitarian Conditions->Coping Mechanisms',
       'Humanitarian Conditions->Living Standards',
       'Humanitarian Conditions->Number Of People In Need',
       'Humanitarian Conditions->Physical And Mental Well Being',
       'Impact->Driver/Aggravating Factors', 'Impact->Impact On People',
       'Impact->Impact On Systems, Services And Networks',
       'Impact->Number Of People Affected',
       'Priority Interventions->Expressed By Humanitarian Staff',
       'Priority Interventions->Expressed By Population',
       'Priority Needs->Expressed By Humanitarian Staff',
       'Priority Needs->Expressed By Population'], dtype=object)

In [21]:
df_sample["sectors_transformed"] = list(mlb_sectors.transform(list(df_sample["sectors"])))
df_sample["sectors_pred_transformed"] = list(mlb_sectors.transform(list(df_sample["sectors_pred"])))
len(df_sample["sectors_transformed"]), len(df_sample["sectors_pred_transformed"])

(150, 150)

In [22]:
df_sample["subpillars_1d_transformed"] = list(mlb_subpillars1d.transform(list(df_sample["subpillars_1d"])))
df_sample["subpillars_1d_pred_transformed"] = list(mlb_subpillars1d.transform(list(df_sample["subpillars_1d_pred"])))

In [23]:
df_sample["subpillars_2d_transformed"] = list(mlb_subpillars2d.transform(list(df_sample["subpillars_2d"])))
df_sample["subpillars_2d_pred_transformed"] = list(mlb_subpillars2d.transform(list(df_sample["subpillars_2d_pred"])))

In [49]:
for project_group_gt, project_group_pred in zip(
    list(df_sample.groupby("project_id")["sectors_transformed"]),
    list(df_sample.groupby("project_id")["sectors_pred_transformed"])):
    #print(list(project_group_gt[1]))
    #print(project_group_pred)
    p, r, f1s, s = precision_recall_fscore_support(list(project_group_gt[1]), list(project_group_pred[1]), average="macro", zero_division=0)
    print(f"Project ID {project_group_gt[0]}: -> Sector level: Precision: {p}, Recall: {r}, F1Score: {f1s}, Support: {s}")
    


Project ID 2028.0: -> Sector level: Precision: 0.8269162210338682, Recall: 0.8424242424242423, F1Score: 0.812258953168044, Support: None
Project ID 2098.0: -> Sector level: Precision: 0.5215728715728716, Recall: 0.6070707070707071, F1Score: 0.5317001180637545, Support: None
Project ID 2170.0: -> Sector level: Precision: 0.4893048128342246, Recall: 0.6480716253443526, F1Score: 0.5437229437229437, Support: None


In [25]:
# Results per tag in Sector group
results_sectors = precision_recall_fscore_support(list(df_sample["sectors_transformed"]), list(df_sample["sectors_pred_transformed"]), zero_division=0)

In [26]:
# Note that these are precisions for each sector tag e.g. Agriculture, Nutrition, Cross etc.
# First list is the precision for each sector tag
# Second list is the recall for each sector tag
# Third list is the f1score for each sector tag
results_sectors

(array([0.5       , 0.45833333, 0.78947368, 0.76923077, 0.73076923,
        0.44444444, 0.75      , 0.2       , 0.5625    , 0.85714286,
        0.875     ]),
 array([0.33333333, 0.57894737, 0.9375    , 0.76923077, 0.82608696,
        0.88888889, 1.        , 0.5       , 0.69230769, 0.75      ,
        0.875     ]),
 array([0.4       , 0.51162791, 0.85714286, 0.76923077, 0.7755102 ,
        0.59259259, 0.85714286, 0.28571429, 0.62068966, 0.8       ,
        0.875     ]),
 array([ 3, 19, 16, 13, 46,  9,  6,  2, 13,  8, 16]))

In [43]:
print("Sector - Precision - calculated for all projects")
print(list((tag, value) for tag, value in zip(sectors, results_sectors[0])))

Sector - Precision - calculated for all projects
[('Agriculture', 0.5), ('Cross', 0.4583333333333333), ('Education', 0.7894736842105263), ('Health', 0.7692307692307693), ('Livelihoods', 0.7307692307692307), ('Logistics', 0.4444444444444444), ('Protection', 0.75), ('Shelter', 0.2), ('WASH', 0.5625), ('Food Security', 0.8571428571428571), ('Nutrition', 0.875)]


In [44]:
print("Sector - Recall - calculated for all projects")
print(list((tag, value) for tag, value in zip(sectors, results_sectors[1])))

Sector - Recall - calculated for all projects
[('Agriculture', 0.3333333333333333), ('Cross', 0.5789473684210527), ('Education', 0.9375), ('Health', 0.7692307692307693), ('Livelihoods', 0.8260869565217391), ('Logistics', 0.8888888888888888), ('Protection', 1.0), ('Shelter', 0.5), ('WASH', 0.6923076923076923), ('Food Security', 0.75), ('Nutrition', 0.875)]


In [45]:
print("Sector - F1Score - calculated for all projects")
print(list((tag, value) for tag, value in zip(sectors, results_sectors[2])))

Sector - F1Score - calculated for all projects
[('Agriculture', 0.4), ('Cross', 0.5116279069767442), ('Education', 0.8571428571428572), ('Health', 0.7692307692307693), ('Livelihoods', 0.7755102040816326), ('Logistics', 0.5925925925925926), ('Protection', 0.8571428571428571), ('Shelter', 0.28571428571428575), ('WASH', 0.6206896551724138), ('Food Security', 0.7999999999999999), ('Nutrition', 0.875)]


In [27]:
# Average over all the projects
avg_precision_sector, avg_recall_sector, avg_f1score_sector, avg_support_sector = precision_recall_fscore_support(
    list(df_sample["sectors_transformed"]), list(df_sample["sectors_pred_transformed"]),
    average="macro", zero_division=0
)
avg_precision_sector, avg_recall_sector, avg_f1score_sector, avg_support_sector

(0.6306267562846509, 0.7410268189730431, 0.667695557095832, None)

In [28]:
# Average over all the projects
avg_precision_subpillars1d, avg_recall_subpillars1d, avg_f1score_subpillars1d, avg_support_subpillars1d = precision_recall_fscore_support(
    list(df_sample["subpillars_1d_transformed"]), list(df_sample["subpillars_1d_pred_transformed"]),
    average="macro", zero_division=0
)
avg_precision_subpillars1d, avg_recall_subpillars1d, avg_f1score_subpillars1d, avg_support_subpillars1d

(0.2720615243342516, 0.260097730552276, 0.26212266969842724, None)

In [29]:
# Average over all the projects
avg_precision_subpillars2d, avg_recall_subpillars2d, avg_f1score_subpillars2d, avg_support_subpillars2d = precision_recall_fscore_support(
    list(df_sample["subpillars_2d_transformed"]), list(df_sample["subpillars_2d_pred_transformed"]),
    average="macro", zero_division=0
)
avg_precision_subpillars2d, avg_recall_subpillars2d, avg_f1score_subpillars2d, avg_support_subpillars2d

(0.4197691197691198, 0.550723110547672, 0.4617275308633557, None)

In [30]:
df_sample["sector_completely_matched"] = list(map(completely_matched_tags, list(zip(np.array(df_sample["sectors_transformed"]), np.array(df_sample["sectors_pred_transformed"])))))
df_sample["sector_completely_matched"] /= len(mlb_sectors.classes_)
df_sample["sector_missing"] = list(map(missing_tags, list(zip(np.array(df_sample["sectors_transformed"]), np.array(df_sample["sectors_pred_transformed"])))))
df_sample["sector_missing"] /= len(mlb_sectors.classes_)
df_sample["sector_wrong"] = list(map(wrong_tags, list(zip(np.array(df_sample["sectors_transformed"]), np.array(df_sample["sectors_pred_transformed"])))))
df_sample["sector_wrong"] /= len(mlb_sectors.classes_)

In [32]:
df_sample["subpillars_1d_completely_matched"] = list(map(completely_matched_tags, list(zip(np.array(df_sample["subpillars_1d_transformed"]), np.array(df_sample["subpillars_1d_pred_transformed"])))))
df_sample["subpillars_1d_completely_matched"] /= len(mlb_subpillars1d.classes_)
df_sample["subpillars_1d_missing"] = list(map(missing_tags, list(zip(np.array(df_sample["subpillars_1d_transformed"]), np.array(df_sample["subpillars_1d_pred_transformed"])))))
df_sample["subpillars_1d_missing"] /= len(mlb_subpillars1d.classes_)
df_sample["subpillars_1d_wrong"] = list(map(wrong_tags, list(zip(np.array(df_sample["subpillars_1d_transformed"]), np.array(df_sample["subpillars_1d_pred_transformed"])))))
df_sample["subpillars_1d_wrong"] /= len(mlb_subpillars1d.classes_)

In [33]:
df_sample["subpillars_2d_completely_matched"] = list(map(completely_matched_tags, list(zip(np.array(df_sample["subpillars_2d_transformed"]), np.array(df_sample["subpillars_2d_pred_transformed"])))))
df_sample["subpillars_2d_completely_matched"] /= len(mlb_subpillars2d.classes_)
df_sample["subpillars_2d_missing"] = list(map(missing_tags, list(zip(np.array(df_sample["subpillars_2d_transformed"]), np.array(df_sample["subpillars_2d_pred_transformed"])))))
df_sample["subpillars_2d_missing"] /= len(mlb_subpillars2d.classes_)
df_sample["subpillars_2d_wrong"] = list(map(wrong_tags, list(zip(np.array(df_sample["subpillars_2d_transformed"]), np.array(df_sample["subpillars_2d_pred_transformed"])))))
df_sample["subpillars_2d_wrong"] /= len(mlb_subpillars2d.classes_)

In [34]:
df_sample.head(5)

,entry_id,excerpt,analysis_framework_id,lead_id,project_id,verified,sectors,subpillars_2d,subpillars_1d,geo_location,...,subpillars_2d_pred_transformed,sector_completely_matched,sector_missing,sector_wrong,subpillars_1d_completely_matched,subpillars_1d_missing,subpillars_1d_wrong,subpillars_2d_completely_matched,subpillars_2d_missing,subpillars_2d_wrong
0,174380,"[6th september,Overall Syria] Riza said there ...",1306,40125.0,2028.0,False,[Health],[Humanitarian Conditions->Physical And Mental ...,[Casualties->Dead],['Syrian Arab Republic'],...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",1.000000,0.000000,0.000000,1.0,0.0,0.0,1.000000,0.000000,0.000000
1,224488,"[December 21, As-Suwayda] Fear of the Coronavi...",1306,46147.0,2028.0,False,"[Protection, Education]",[Humanitarian Conditions->Physical And Mental ...,[],['As-Sweida'],...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, ...",0.818182,0.090909,0.090909,1.0,0.0,0.0,0.833333,0.055556,0.111111
2,245365,"[November 2020, North-west Syria] By November,...",1306,47945.0,2028.0,False,[],[],[Covid-19->Cases],['Northwest'],...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.909091,0.000000,0.090909,1.0,0.0,0.0,0.944444,0.000000,0.055556
3,326857,"(Idlib, 03/2021) The Director General of the G...",1306,51351.0,2028.0,False,[WASH],"[Impact->Impact On Systems, Services And Netwo...",[],[],...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, ...",1.000000,0.000000,0.000000,1.0,0.0,0.0,0.944444,0.000000,0.055556
4,164076,Trend analysis and overview: The number of con...,1306,38063.0,2028.0,False,[Health],[Humanitarian Conditions->Physical And Mental ...,[Casualties->Dead],['Syrian Arab Republic'],...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",1.000000,0.000000,0.000000,1.0,0.0,0.0,1.000000,0.000000,0.000000


In [35]:
# Per project mean values for sectors
sectors_completely_matched = df_sample.groupby(["project_id"])["sector_completely_matched"].mean()
sectors_missing = df_sample.groupby(["project_id"])["sector_missing"].mean()
sectors_wrong = df_sample.groupby(["project_id"])["sector_wrong"].mean()

sectors_completely_matched, sectors_missing, sectors_wrong

(project_id
 2028.0    0.958182
 2098.0    0.927273
 2170.0    0.945455
 Name: sector_completely_matched, dtype: float64,
 project_id
 2028.0    0.012727
 2098.0    0.023636
 2170.0    0.021818
 Name: sector_missing, dtype: float64,
 project_id
 2028.0    0.029091
 2098.0    0.049091
 2170.0    0.032727
 Name: sector_wrong, dtype: float64)

In [36]:
# Per project mean values for subpillars 1d
subpillars_1d_completely_matched = df_sample.groupby(["project_id"])["subpillars_1d_completely_matched"].mean()
subpillars_1d_missing = df_sample.groupby(["project_id"])["subpillars_1d_missing"].mean()
subpillars_1d_wrong = df_sample.groupby(["project_id"])["subpillars_1d_wrong"].mean()

subpillars_1d_completely_matched, subpillars_1d_missing, subpillars_1d_wrong

(project_id
 2028.0    0.992121
 2098.0    0.990909
 2170.0    0.989091
 Name: subpillars_1d_completely_matched, dtype: float64,
 project_id
 2028.0    0.005455
 2098.0    0.003636
 2170.0    0.004242
 Name: subpillars_1d_missing, dtype: float64,
 project_id
 2028.0    0.002424
 2098.0    0.005455
 2170.0    0.006667
 Name: subpillars_1d_wrong, dtype: float64)

In [37]:
# Per project mean values for subpillars 2d
subpillars_2d_completely_matched = df_sample.groupby(["project_id"])["subpillars_2d_completely_matched"].mean()
subpillars_2d_missing = df_sample.groupby(["project_id"])["subpillars_2d_missing"].mean()
subpillars_2d_wrong = df_sample.groupby(["project_id"])["subpillars_2d_wrong"].mean()

subpillars_2d_completely_matched, subpillars_2d_missing, subpillars_2d_wrong

(project_id
 2028.0    0.947778
 2098.0    0.941111
 2170.0    0.950000
 Name: subpillars_2d_completely_matched, dtype: float64,
 project_id
 2028.0    0.011111
 2098.0    0.017778
 2170.0    0.025556
 Name: subpillars_2d_missing, dtype: float64,
 project_id
 2028.0    0.041111
 2098.0    0.041111
 2170.0    0.024444
 Name: subpillars_2d_wrong, dtype: float64)

In [38]:
# For Sectors
confusion_matrix = multilabel_confusion_matrix(list(df_sample["sectors_transformed"]), list(df_sample["sectors_pred_transformed"]))
confusion_matrix

array([[[146,   1],
        [  2,   1]],

       [[118,  13],
        [  8,  11]],

       [[130,   4],
        [  1,  15]],

       [[134,   3],
        [  3,  10]],

       [[ 90,  14],
        [  8,  38]],

       [[131,  10],
        [  1,   8]],

       [[142,   2],
        [  0,   6]],

       [[144,   4],
        [  1,   1]],

       [[130,   7],
        [  4,   9]],

       [[141,   1],
        [  2,   6]],

       [[132,   2],
        [  2,  14]]])